## Prerequisite
- The PNG image in the input folder must be __*1-bit image*__ (means only black & white)
- The naming must follow __*R12EWRA_ft_1700_6870_SEXP_OHMM_lg_0.06_6.png*__ convention to match the regex


In [ ]:
from PIL import Image
import math, re, json
import pandas as pd
import numpy as np
from os import listdir, path

with open('../elvconfig.json') as cfg_data:
    cfg = json.load(cfg_data)

inputpath = cfg['inputpath']
outputpath = cfg['outputpath']

step_size = 0.2 # in depth unit
multiples = 1 # lengthening of log, use 1 for .las
img_ext = ".png"

img_files = [(inputpath + f) for f in listdir(inputpath) if img_ext in f]
for img_file in img_files:
    # Logarithmic R12EWRA_ft_1700_6870_SEXP_OHMM_lg_0.06_6
    # Linear      R13EWRB_ft_1700_6870_SNPE_BLE_ln_0_15
    pat = r'(.*.)_(\w+)_(\d+)_(\d+)_(\w+)_(\w+)_(\w+)_([+-]?\d+(?:\.\d+)?)_([+-]?\d+(?:\.\d+)?).png'
    
    lasfile = re.search(pat,img_file).group(1)
    depth_unit = re.search(pat,img_file).group(2)
    y_min = int(re.search(pat,img_file).group(3))
    y_max = int(re.search(pat,img_file).group(4))
    curve = re.search(pat,img_file).group(5)
    unit = re.search(pat,img_file).group(6)
    scale = re.search(pat,img_file).group(7)
    x_min = float(re.search(pat,img_file).group(8))
    x_max = float(re.search(pat,img_file).group(9))
    
    y_is_reverse = y_max < y_min
    
    if y_is_reverse:
        y_max,y_min = y_min,y_max
    print("{}, {}, {}, {}, {}, {}, {}, {}, {}".format(lasfile,depth_unit,y_min,y_max,curve,unit,scale,x_min,x_max))
    
    output_file = img_file[:-4] + ".txt"
    img = Image.open(img_file,mode='r')
      
    if (scale == 'lg'):
        width = int(abs(math.log10(x_min) - math.log10(x_max)))
    elif (scale == 'ln'):
        width = x_max - x_min
        
    height = y_max - y_min
    steps_count = math.ceil(height/step_size)
    height = step_size *  steps_count
    y_max = y_min + height #main calculation time
    img = img.resize((img.width,abs(steps_count * multiples)))
    pixels_img = [(y,x)
         for y in range(img.height)
         for x in range(img.width)
         if img.getpixel((x,y)) == 0]

    df = pd.DataFrame(pixels_img)
    dfx = df.groupby([0]).median().rename(columns = {1:'median'})
    dfx['max'] = df.groupby([0]).max()
    dfx['min'] = df.groupby([0]).min()
    dfx['selected'] = np.nan
    dfx['pixel_row'] = dfx.index
    dfx = dfx[dfx.index % multiples == 0]
    dfx = dfx.reset_index(drop=True)
    for index, row in dfx.iterrows():
        if index != 0 and index < dfx.shape[0] - 1:
            before = dfx['median'][index - 1]       
        else: 
            before = dfx['median'][index]  

        now = dfx['median'][index]

        if (before == now):
            dfx.set_value(index,'selected',dfx['median'][index])
        elif (before > now):
            dfx.set_value(index,'selected',dfx['min'][index])
        elif (before < now):
            dfx.set_value(index,'selected',dfx['max'][index])
    
    dfx['Depth'] = (dfx['pixel_row'] * (step_size/multiples) + y_min)
    dfx['xAdj'] = (dfx['selected'] * (width/img.width))
    
    if (scale == 'lg'):
        dfx[curve] = (x_min*10**dfx['xAdj']) # converting linear to log
    elif (scale == 'ln'):
        dfx[curve] = dfx['xAdj'] + x_min
        
    if(y_is_reverse):
        dfx[curve] = dfx[curve].values[::-1]
        
    dfx_output = dfx[['Depth',curve]]
    
    df_unit = pd.DataFrame([[depth_unit,unit]],columns=['Depth',curve])
    
    df_unit.to_csv(path_or_buf=path.join(outputpath,output_file),mode="a", header=True,sep="\t",index=False)
    dfx_output.to_csv(path_or_buf=path.join(outputpath,output_file),mode="a", header=False,sep="\t",index=False)

print("Finished")
       